# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

In [147]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
#from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
from detoxify import Detoxify

In [148]:
try:
    data = pd.read_csv('C:/project/project13/toxic_comments.csv', on_bad_lines='skip')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv', on_bad_lines='skip')

## Подготовка данных

In [149]:
#data.info()
#data.toxic.unique()
data.head(3)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


### С данными все в порядке: Целевой признак в числовом формате, уникальные значения 0 и 1. Необходимо только удалить столбец 'Unnamed: 0'.

In [150]:
data = data.drop(columns='Unnamed: 0')

## Проведем предобработку данных


### Выделим некоторые признаки из текста, которые могут иметь значение. Тут следует отметить, что оригинальная задача решалась на кейсе на определение авторства текстов. В нашем случае есть смысл отдельно посчитать восклицательные знаки и символы, которые используют для закрытия части нецензурных слов ($#*%).

In [151]:
stopWords = set(stopwords.words('english'))

#напишем функцию, которая будет извлекать признаки из текста
# https://www.kaggle.com/code/gooogr/a-deep-dive-into-sklearn-pipelines-forked/notebook
def processing(df):
    #приведение к нижнему регистру и удаление пунктуации
    df['processed'] = df['text'].apply(lambda x: " ".join(re.sub(r"[^a-zA-Z']", " ", x.lower()).split())) 
    #длина предложения (символов) 
    df['length'] = df['processed'].apply(lambda x: len(x))
    #количество слов
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    #количество слов, за исключением stopwords - слов, не несущих смысловой накрузки
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #считаем восклицательные знаки Exclamation point
    df['exclamation'] = df['text'].apply(lambda x: x.count('!'))
    #Why Do We Use Symbols To Censor Swearwords
    df['simbols'] = df['text'].apply(lambda x: x.count('$')+x.count('#')+x.count('*')+x.count('%')) #$#*%

    return(df)

data = processing(data)

data.head(3)

,text,toxic,processed,length,words,words_not_stopword,exclamation,simbols
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,248,43,24,0,0
1,D'aww! He matches this background colour I'm s...,0,d'aww he matches this background colour i'm se...,88,14,11,1,0
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i'm really not trying to edit war it's...,229,42,22,0,0


In [152]:
df = data.copy() #для эксперимента с модулем Detoxify

### Лемматизируем текст в колонке 'processed', удалим уже ненужную колонку 'text'

In [153]:
%%time
# Функция возвращает тэг типа слова в wordnet (nltk.pos_tag выдает какие-то свои типы)
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

def lemmatize(text): # в функцию подается ощищенный текст, на выходе получаем лемматизированный текст по 
#всем типам слов (lemmatizer + POS)
    lemmatizer = WordNetLemmatizer()
    # tokenize the sentence and find the POS tag for each token
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))   
    # As you may have noticed, the above pos tags are a little confusing.
    # we use our own pos_tagger function to make things simpler to understand.
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)
    return lemmatized_sentence

data['processed'] = data.processed.apply(lemmatize)
data = data.drop(columns='text')

data.head(3)

Wall time: 15min 8s


,toxic,processed,length,words,words_not_stopword,exclamation,simbols
0,0,explanation why the edits make under my userna...,248,43,24,0,0
1,0,d'aww he match this background colour i 'm see...,88,14,11,1,0
2,0,hey man i 'm really not try to edit war it 's ...,229,42,22,0,0


### Как мы видим, признаки в нашем датасете неоднородны. Числовые хорошо бы отмаштабировать, а на основе текста создать векторы Tfid. 

### Разобьем выборки на тренировочную, валидационную и тестовую. Валидационную выборку можно было бы не выделять (сделать кросс-валидацию), но на ней надо будет подобрать оптимальный порог для лучшего F1.

In [154]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=12345)
data_train, data_val = train_test_split(data_train, test_size=0.25, random_state=12345)

### Сама схема для Pipeline следующая: 
1. Преобразуем текстовое поле
2. Преобразуем числовые поля
3. Объединим преобразованные поля в один датасет
4. На тренировочной выборке определим оптимальные параметры через кросс-валидацию, максимизируя Auc-Roc
5. На тренировочной и валидационных выборках определим порог для максимального F1 для этих параметров
6. Проверим нашу модель на тестовой выборке

### Для того, чтобы выбрать нужные поля для преобразования в рамках Pipeline, необходимо создать свой собственный класс, который будет возвращать колонку с заданным типом данных.

In [155]:
class DataFrameSelector(BaseEstimator, TransformerMixin): #судя по всему, класс наследует опеределенные параметры
    # и методы от классов BaseEstimator, TransformerMixin, которые позволяют использовать его в Pipeline
    """
     Transformer to select a part of the data frame to perform additional transformations on
     """
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names]

In [156]:
str_feature = 'processed'
num_features_list = ['length', 'words', 'words_not_stopword', 'exclamation', 'simbols']

### Векторизируем текст

In [157]:
text = Pipeline([
                ('selector', DataFrameSelector(str_feature)),
                ('tfidf', TfidfVectorizer(stop_words='english'))
            ])

text.fit_transform(data_train)

<95574x114641 sparse matrix of type '<class 'numpy.float64'>'
	with 2204859 stored elements in Compressed Sparse Row format>

### Масштабируем числовые признаки

In [158]:
num_feats = Pipeline([
    ('selector', DataFrameSelector(num_features_list)),
    ('standard', StandardScaler())
])

num_feats.fit_transform(data_train)

array([[-0.19478033, -0.21112447, -0.16722706, -0.0255081 , -0.10662224],
       [ 0.22583871,  0.25118041,  0.1828991 , -0.0255081 , -0.10662224],
       [-0.47103437, -0.48247734, -0.46207013, -0.0255081 , -0.10662224],
       ...,
       [ 0.19375759,  0.16072946,  0.25660987, -0.0255081 , -0.10662224],
       [-0.5904474 , -0.59302851, -0.59106398, -0.0255081 , -0.10662224],
       [-0.38013788, -0.40207649, -0.36993167, -0.0255081 , -0.10662224]])

### Объединяем в один датасет.

In [159]:
#The feature union itself is not a pipeline, it's just a union, so you need to do one more step to make it useable: 
#pass it to a pipeline, with the same structure, an array of tuples, with the simple (name, object) format

feats = FeatureUnion([
    ('text', text),
    ('num_feats', num_feats)
])

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(data_train)

<95574x114646 sparse matrix of type '<class 'numpy.float64'>'
	with 2682729 stored elements in Compressed Sparse Row format>

## Подберем оптимальную модель

### Подберем параметры для логической регрессии, которая традиционно дает хорошие результаты для текстов.
Выбор модели основан на этой статье https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

#### Добавим в Pipeline логическую регрессию и выведем доступные для настройки параметры для всего пайплайна.

In [160]:
%%time
pipeline = Pipeline([
    ('features',feats),
    ('lgr', LogisticRegression(max_iter=1000)), # задал max_iter здесь, т.к. на дефолтных 100 выдает ошибку
])

pipeline.get_params().keys()

Wall time: 0 ns


dict_keys(['memory', 'steps', 'verbose', 'features', 'lgr', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text', 'features__num_feats', 'features__text__memory', 'features__text__steps', 'features__text__verbose', 'features__text__selector', 'features__text__tfidf', 'features__text__selector__attribute_names', 'features__text__tfidf__analyzer', 'features__text__tfidf__binary', 'features__text__tfidf__decode_error', 'features__text__tfidf__dtype', 'features__text__tfidf__encoding', 'features__text__tfidf__input', 'features__text__tfidf__lowercase', 'features__text__tfidf__max_df', 'features__text__tfidf__max_features', 'features__text__tfidf__min_df', 'features__text__tfidf__ngram_range', 'features__text__tfidf__norm', 'features__text__tfidf__preprocessor', 'features__text__tfidf__smooth_idf', 'features__text__tfidf__stop_words', 'features__text__tfidf__strip_accents', 'features__text__tfidf__sublinear_tf', 'features__

#### Выберем самые интересные параметры для перебора и определим те, которые максимизируют 'roc_auc'

In [161]:
%%time
hyperparameters = { 'features__text__tfidf__max_features': [1000, None],
                    'features__text__tfidf__ngram_range': [(1,1), (1,2)],
                    'lgr__class_weight': [None, 'balanced'],
                    'lgr__C': [0.1, 1, 10]
                  }
clf = GridSearchCV(pipeline, hyperparameters, cv=3, scoring='roc_auc', n_jobs=-1)
 
# Fit and tune model
clf.fit(data_train, data_train['toxic'])
clf.best_params_

#Wall time: 30min 41s
#{'features__text__tfidf__max_features': None,
# 'features__text__tfidf__ngram_range': (1, 2),
# 'lgr__C': 10,
# 'lgr__class_weight': None}

Wall time: 31min 2s


{'features__text__tfidf__max_features': None,
 'features__text__tfidf__ngram_range': (1, 2),
 'lgr__C': 10,
 'lgr__class_weight': None}

In [162]:
#refitting on entire training data using best settings
clf.refit
probabilities_valid = clf.predict_proba(data_val)
probabilities_one_valid = probabilities_valid[:, 1]

In [163]:
f1_max = 0
threshold_opt = None

for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    f1 = f1_score(predicted_valid, data_val['toxic'])
    if f1>f1_max:
        f1_max=f1
        threshold_opt = threshold
        
print (f1_max, threshold_opt) 
# 0.7749523204068658 0.28

0.7749523204068658 0.28


In [164]:
probabilities_test = clf.predict_proba(data_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.28
f1_score(predicted_test, data_test['toxic'])
#0.7889471176750834

0.7889471176750834

#### Логическая регрессия показала на тестовой выборке F1 0.79 (на валидационной 0.775) при пороге 0.28 и следующих параметрах:
1. TfidfVectorizer: max_features: None, ngram_range: (1, 2)
2. LogisticRegression(max_iter=1000): C: 10, class_weight: None

### Сравним достигнутый показатель качества с предсказаниями на основе модуля Detoxify (toxic Bert). 
Модуль на входе получает ощищенный текст, назад возвращает некоторые оценки токсичности, основанные на 7 параметрах: 'toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit'.

#### Сократим размер датасета до 10000 записей, к сожалению, полный будет обрабатываться значительное время.

In [165]:
#https://github.com/unitaryai/detoxify
# GPU https://www.kaggle.com/code/sorenj/scoring-comments-using-unitaryai-detoxify/notebook
df = df.sample(10000, random_state=12345)

In [166]:
predictor = Detoxify('unbiased')

In [167]:
def new_features(x):
    prediction = predictor.predict(x['processed'])
    return prediction['toxicity'], prediction['severe_toxicity'], prediction['obscene'], prediction['identity_attack'], prediction['insult'], prediction['threat'], prediction['sexual_explicit']

In [168]:
%%time
df_features = df.apply(new_features, axis=1, result_type='expand')
df_features.columns = ['toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']

Wall time: 55min 12s


#### После преобразования столбца с текстом, у нас получился следующий датасет с признаками:

In [169]:
df_features.head(3)

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
109486,0.002178,0.000002,0.000042,0.000863,0.000376,0.000038,0.000020
104980,0.128936,0.000047,0.009342,0.000694,0.027453,0.000345,0.019332
82166,0.000613,0.000002,0.000034,0.000080,0.000107,0.000029,0.000022


#### Разобьем выборку на тренировочную, валидационную и тестовую, подберем оптимальные параметры логической регрессии, максимизируем F1.

In [170]:
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df['toxic'], test_size=0.2, random_state=12345)
df_features_train, df_features_val, df_target_train, df_target_val = train_test_split(df_features_train, df_target_train, test_size=0.25, random_state=12345)

In [171]:
%%time
model = LogisticRegression(max_iter=1000)
hyperparameters = {
                    'class_weight': [None, 'balanced'],
                    'C': [0.1, 1, 10]
                  }

clf = GridSearchCV(model, param_grid = hyperparameters, cv=3, scoring='roc_auc', n_jobs=-1)
 
# Fit and tune model
clf.fit(df_features_train, df_target_train)
clf.best_params_

Wall time: 11.4 s


{'C': 0.1, 'class_weight': 'balanced'}

In [172]:
#refitting on entire training data using best settings
clf.refit
probabilities_valid = clf.predict_proba(df_features_val)
probabilities_one_valid = probabilities_valid[:, 1]

In [173]:
f1_max = 0
threshold_opt = None

for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    f1 = f1_score(predicted_valid, df_target_val)
    if f1>f1_max:
        f1_max=f1
        threshold_opt = threshold
        
print (f1_max, threshold_opt) 

0.8534704370179949 0.8200000000000001


In [174]:
probabilities_test = clf.predict_proba(df_features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.2
f1_score(predicted_test, df_target_test)

0.7607573149741824

#### На валидационной выборке удалось достичь учень хорошего уровня F1: 0.85. При этом, показатели на тестовой выборке существенно ухудшились, всего 0.76. Полагаю, что это связано с недостаточностью выборки, и по полным данным можно выйти на F1 0.84- 0.85.

# Выводы
1. Нам удалось постоить модель с параметрами F1 на тестовой выборке, превышающими 0.75 (0.79(0.775 - валидация)).
2. Модуль Detoxify потенциально показывает существенно больший F1 (до 0.85), но требует обработки датасета на существенно бОльших мощностях. При обработке всего датасета с GPU на Google Colab, модель достигла F1 0.884 на тестовой выборке (0.917 на валидационной)